### 1. Problem statement
- We are tasked by a well-known video streaming company to predict its subscribers' taste in movies, rating from 0 to 5.
- This notebook is to create a feasbility model using Stacked Auto-encoder

### 2. Import packages

In [1]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.optim as optim
import torch.utils.data
from torch.autograd import Variable

### 3. Read source data

In [2]:
movies = pd.read_csv('ml-1m/movies.dat', sep = '::', header = None, engine = 'python', encoding = 'latin-1')
users = pd.read_csv('ml-1m/users.dat', sep = '::', header = None, engine = 'python', encoding = 'latin-1')
ratings = pd.read_csv('ml-1m/ratings.dat', sep = '::', header = None, engine = 'python', encoding = 'latin-1')

In [3]:
movies.head()

,0,1,2
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


In [4]:
users.head()

,0,1,2,3,4
0,1,F,1,10,48067
1,2,M,56,16,70072
2,3,M,25,15,55117
3,4,M,45,7,02460
4,5,M,25,20,55455


ratings: user ID, movie ID, rating, timestampes

In [5]:
ratings.head()

,0,1,2,3
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


#### 3.1 Read in train and test sets

training_set: 
- user ID, movie ID, rating, timestampes

In [6]:
##create training and test set data
# here only take u1.base but there are another u2 u3 u4 u5 base files
training_set = pd.read_csv('ml-100k/u1.base', delimiter = '\t', header = None)
print(training_set.head(3))

   0  1  2          3
0  1  1  5  874965758
1  1  2  3  876893171
2  1  3  4  878542960


In [7]:
##convert it to array
training_set = np.array(training_set, dtype = 'int')

In [8]:
training_set.shape

(80000, 4)

In [9]:
test_set = pd.read_csv('ml-100k/u1.test', delimiter = '\t', header = None)
##convert it to array
test_set = np.array(test_set, dtype = 'int')

In [10]:
test_set.shape

(20000, 4)

### 4. Data structure creation

#### 4.1 Create data strucutre

In [11]:
#take max users id in train and test data
nb_users = int(max(max(training_set[:,0]), max(test_set[:,0])))
nb_movies = int(max(max(training_set[:,1]), max(test_set[:,1])))

In [12]:
print('Num of users: ', nb_users, '\nNum of movies: ', nb_movies)

Num of users:  943 
Num of movies:  1682


In [13]:
def convert(data):
    new_data = []
    for id_users in range(1, nb_users + 1):
        ##id of movies that is rated by current users
        id_movies = data[:,1][data[:,0] == id_users]
        
        ##rate of movies that is given by current user
        id_ratings = data[:,2][data[:,0] == id_users]
        
        #inialize ratings for all movies
        #set 0 for movies that are not rated by current users
        ratings = np.zeros(nb_movies)
        #movie id starts from 1, 1st movie will be 1st element in rating with index as 0
        ratings[id_movies - 1] = id_ratings
        new_data.append(list(ratings))
    return new_data

In [14]:
training_set = convert(training_set)
test_set = convert(test_set)

#### 4.2 convert data into Torch tensor

In [15]:
training_set = torch.FloatTensor(training_set)
test_set = torch.FloatTensor(test_set)

In [16]:
training_set

tensor([[5., 3., 4.,  ..., 0., 0., 0.],
        [4., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [5., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 5., 0.,  ..., 0., 0., 0.]])

In [17]:
test_set

tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]])

### 5. SAE architecture creation

In [18]:
class SAE(nn.Module):
    def __init__(self, ):
        #allow to inhert all classes and methods of parent class
        super(SAE, self).__init__()
        #num of features from input: num of movies, 20 nodes in first hidden layer
        self.fc1 = nn.Linear(nb_movies, 20)
        self.fc2 = nn.Linear(20, 10)
        #start to decoding
        self.fc3 = nn.Linear(10, 20)
        self.fc4 = nn.Linear(20, nb_movies)
        self.activation = nn.Sigmoid()
        
    #input vector (movies rating) for a specific users
    def forward(self, x):
        #apply activaton fuc on first encoding layer
        #return first encoded vector
        x = self.activation(self.fc1(x))
        x = self.activation(self.fc2(x))
        x = self.activation(self.fc3(x))
        #no need to activate the vector
        #output the reconstrctured vector
        x = self.fc4(x)
        return x

### 6. Model training

In [19]:
sae = SAE()
#Create loss fucn object
criterion = nn.MSELoss()
#create optimizer object
#parameters of all auto-encoders defined in the class
optimizer = optim.RMSprop(sae.parameters(), lr = 0.01, weight_decay = 0.5)

In [20]:
# #train the SAE using pytorch only codes

#loop all epochs
nb_epoch = 200
for epoch in range(1, nb_epoch + 1):
    train_loss = 0
    #exclude users who did not rate any movies
    #define a float
    s = 0.
    #loop through each users
    for id_user in range(nb_users):
        #get all rating for current user from training_set
        #nn does not take single dimension vector, so add a batch dimension
        #a batch of sinlge inptu vector, update weigths after each vector
        input = Variable(training_set[id_user]).unsqueeze(0)
        #create target by copying input
        target = input.clone()
        #only look at users who rated at least 1 movie
        if torch.sum(target.data > 0) > 0:
            #get output from the network, a vector of predicted value
            output = sae(input)
            #do not compute gradient with respect to target
            target.require_grad = False
            #don't account the output whose initial input is 0
            output[target == 0] = 0
            loss = criterion(output, target)
            #make demonitor is not zero, to add a small number
            mean_corrector = nb_movies / float(torch.sum(target.data>0) + 1e-10)
            #backward method to determine which direction 
            loss.backward()
            #access the data of loss object .data[0]
            #adjust the loss to compute relevant mean for all movies for current user
            train_loss += np.sqrt(loss.data * mean_corrector)
            s += 1.
            #apply optimizer to update weights, decides the amount of weight udpates
            optimizer.step()
    if epoch % 10 == 0:
        print('epoch: '+str(epoch) + ' loss: ' + str(train_loss/s))


epoch: 10 loss: tensor(1.0197)
epoch: 20 loss: tensor(1.0163)
epoch: 30 loss: tensor(1.0118)
epoch: 40 loss: tensor(0.9933)
epoch: 50 loss: tensor(0.9757)
epoch: 60 loss: tensor(0.9668)
epoch: 70 loss: tensor(0.9520)
epoch: 80 loss: tensor(0.9440)
epoch: 90 loss: tensor(0.9378)
epoch: 100 loss: tensor(0.9333)
epoch: 110 loss: tensor(0.9300)
epoch: 120 loss: tensor(0.9275)
epoch: 130 loss: tensor(0.9243)
epoch: 140 loss: tensor(0.9218)
epoch: 150 loss: tensor(0.9200)
epoch: 160 loss: tensor(0.9175)
epoch: 170 loss: tensor(0.9160)
epoch: 180 loss: tensor(0.9148)
epoch: 190 loss: tensor(0.9133)
epoch: 200 loss: tensor(0.9121)


### 7. SAE testing

#### 7.1 Evaluation

In [21]:
#loop through each users
test_loss = 0
s = 0.
for id_user in range(nb_users):
    #keep using training set
    input = Variable(training_set[id_user]).unsqueeze(0)
    #create target by copying input
    target = Variable(test_set[id_user]).unsqueeze(0)
    #only look at users who rated at least 1 movie
    if torch.sum(target.data > 0) > 0:
        #get output from the network, a vector of predicted value
        output = sae(input)
        #do not compute gradient with respect to target
        target.require_grad = False
        #don't account the output whose initial input is 0
        output[target == 0] = 0
        loss = criterion(output, target)
        #make demonitor is not zero, to add a small number
        mean_corrector = nb_movies/float(torch.sum(target.data > 0) + 1e-10)
        
        
        #access the data of loss object .data[0]
        #adjust the loss to compute relevant mean for all movies for current user
        test_loss += np.sqrt(loss.data*mean_corrector)
        s += 1.
print('test loss: '+str(test_loss/s))

test loss: tensor(0.9521)


#### 7.2 Prediction

In [22]:
user_id = 0
movie_title = movies.iloc[:nb_movies, 1:2]
user_rating = training_set.data.numpy()[user_id, :].reshape(-1,1)
user_target = test_set.data.numpy()[user_id, :].reshape(-1,1)

In [23]:
#to be predicted, target
user_target[user_target>0]

array([5., 3., 5., 5., 3., 4., 4., 3., 2., 3., 4., 2., 4., 5., 4., 3., 4.,
       3., 3., 4., 5., 4., 3., 5., 4., 3., 3., 3., 4., 3., 1., 4., 1., 4.,
       5., 5., 4., 3., 5., 4., 5., 3., 5., 3., 4., 5., 2., 1., 1., 4., 5.,
       1., 5., 5., 3., 3., 1., 4., 3., 4., 5., 3., 4., 4., 1., 1., 2., 2.,
       5., 4., 5., 2., 4., 3., 4., 4., 4., 3., 5., 5., 5., 5., 5., 3., 5.,
       4., 4., 4., 3., 3., 5., 4., 5., 3., 3., 5., 4., 5., 4., 4., 4., 2.,
       4., 3., 3., 1., 5., 4., 5., 2., 3., 4., 5., 4., 4., 3., 2., 5., 4.,
       4., 5., 1., 4., 4., 2., 5., 1., 2., 5., 1., 1., 3., 2., 4., 1., 4.,
       3.], dtype=float32)

In [24]:
user_input = Variable(training_set[user_id]).unsqueeze(0)
# print('training input: ', len(training_set[user_id]), training_set[user_id])
predicted = sae(user_input)
predicted = np.round(predicted.data.numpy().reshape(-1,1), 2)
# print('predicted: \n', len(predicted), predicted)

In [25]:
user_input = user_input.data.numpy().reshape(-1,1)
result_array = np.hstack([movie_title, user_input, user_target, predicted])
result_df = pd.DataFrame(data=result_array, columns=['Movie', 'User input', 'Target Rating', 'Predicted'])

In [26]:
result_df.head()

,Movie,User input,Target Rating,Predicted
0,Toy Story (1995),5.0,0.0,3.94
1,Jumanji (1995),3.0,0.0,3.50
2,Grumpier Old Men (1995),4.0,0.0,3.00
3,Waiting to Exhale (1995),3.0,0.0,3.56
4,Father of the Bride Part II (1995),3.0,0.0,3.42


In [27]:
result_df[result_df['Target Rating'] > 0]

,Movie,User input,Target Rating,Predicted
5,Heat (1995),0.0,5.0,3.90
9,GoldenEye (1995),0.0,3.0,3.97
11,Dracula: Dead and Loving It (1995),0.0,5.0,4.48
13,Nixon (1995),0.0,5.0,3.89
16,Sense and Sensibility (1995),0.0,3.0,3.39
...,...,...,...,...
263,Legends of the Fall (1994),0.0,2.0,2.78
264,Major Payne (1994),0.0,4.0,4.08
265,Little Odessa (1994),0.0,1.0,2.48
266,My Crazy Life (Mi vida loca) (1993),0.0,4.0,2.64
